# **Demand Estimation Exercise**

### Presentation

This exemple is based on the 12th edition of the book **Managerial Economics - Foundations of Business Analysis and Strategy** by Thomas and Maurice (2016).

The data is in the directory, in the file `Data for Checkers Pizza.xlsx`.

In this exercise, the Chechers Pizza wants to set its price. Therefore, is useful estimate the demand for this market. 
<br>The manager estimates the following linear specification of demand using the 24 monthly observations she collected:

---
#### Model

$$ 
Q = a + bP + cM + dP_{Al} + eP_{BMac} 
$$

Where

$Q =$ sales of pizza at Checkers Pizza

$P = $ price of a pizza at Checkers Pizza

$M = $ average annual household income in Westbury

$P_{Al} = $ price of a pizza at Al's Pizza Oven

$P_{BMac} = $ price of a Big Mac at McDonald's

In [1]:
import pandas as pd
from statsmodels.formula.api import ols

---
### Data

In [2]:
data_market = pd.read_excel('Data for Checkers Pizza.xlsx')
data_market.head()

,Observation,Q,P,M,P_Al,P_BMac
0,1,2659,8.65,25500,10.55,1.25
1,2,2870,8.65,25600,10.45,1.35
2,3,2875,8.65,25700,10.35,1.55
3,4,2849,8.65,25970,10.30,1.05
4,5,2842,8.65,25970,10.30,0.95


---
### Regression

We can avoid the `family` especification, because the *defaut* is the Gaussian family.

In [3]:
model_formula = 'Q ~ P + M + P_Al + P_BMac'

model_fit = ols(formula=model_formula,
                data=data_market).fit()

print(model_fit.summary())

                            OLS Regression Results                            
Dep. Variable:                      Q   R-squared:                       0.955
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     101.9
Date:                Thu, 14 Dec 2023   Prob (F-statistic):           1.47e-12
Time:                        16:44:42   Log-Likelihood:                -121.19
No. Observations:                  24   AIC:                             252.4
Df Residuals:                      19   BIC:                             258.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1183.8025    506.298      2.338      0.0

#### Confidence intervals

In [4]:
print(model_fit.conf_int(alpha=0.05))

                    0            1
Intercept  124.108581  2243.496392
P         -241.649087  -185.194716
M            0.065114     0.117063
P_Al        20.202862   182.402851
P_BMac      15.124471   128.565127


This is an important part in regression analysis.

The checking of confidence intervals give us a range of possible efects of each variable on the quantity.
<br>For example, the price coefficient was estimated at `-213.4219`. 
<br>Nonetheless, as an old master used to say, if you want 95% confidence in the shot, you need to consider `-241.649087` as the lower limit and `-185.194716` as the upper limit.

This must be considered in all estimated .
However, for the sake of simplicity (and didactic), we will continue without applying confidence intervals. 

---
### Some Analysis

#### Quantities

The manager decides to calculate the estimated demand elasticities, using the value of the varibles that typify the market.
<br> These values, _for her_, are the **average values of each variable**. We'll call them *variable "bar"*.

At this "typical" point on the estimated demand curve, we can determine the quantity of pizza demanded.

$$ \hat{Q} = \hat{a} + \hat{b}\bar{P} + \hat{c}\bar{M} + \hat{d}\bar{P}_{Al} + \hat{e}\bar{P}_{BMac} $$

For this, we use the `.predict()` method.

For the inputs, we use the mean of the variables. Take a look!

In [5]:
inputs = data_market.mean()
print(inputs)

Observation       12.500000
Q               2784.666667
P                  9.046250
M              26613.208333
P_Al              10.118750
P_BMac             1.145833
dtype: float64


In [6]:
# Estimated average quantity
Q_hat =  model_fit.predict(data_market.mean())[0]
print('Q_hat = ',Q_hat)
print('At these values, the quantity of demanded pizza is, approximatelly, {:.0f}.'.format(Q_hat))

Q_hat =  2784.66666666077
At these values, the quantity of demanded pizza is, approximatelly, 2785.


#### Elasticities

Now, let's estimate the elasticities as follows:

$$ \hat{E} = \hat{b}(\bar{P}/\hat{Q}) $$

$$ \hat{E}_M = \hat{c}(\bar{M}/\hat{Q}) $$

$$ \hat{E}_{XAl} = \hat{d}(\bar{P}_{Al}/\hat{Q}) $$

$$ \hat{E}_{XBMac} = \hat{e}(\bar{P}_{BMac}/\hat{Q}) $$

Where,

$\hat{E} = $ Price elasticity of demand

$\hat{E}_M = $ Income elasticity

$\hat{E}_{XAl} = $ Estimated cross-price elasticity between Al's Pizza price and Checkers' Pizza quantity

$\hat{E}_{XBMac} = $ Estimated cross-price elasticity between Big Mac's price and Checkers' Pizza quantity

In [7]:
# Coefficients
fit = model_fit.params.loc
a_hat = fit['Intercept'] 
b_hat = fit['P']
c_hat = fit['M']
d_hat = fit['P_Al']
e_hat = fit['P_BMac']
# Average value of variables
mean = data_market.describe().loc['mean'].loc
P_bar = mean['P']
M_bar = mean['M']
P_Al_bar = mean['P_Al']
P_BMac_bar = mean['P_BMac']

# Elasticities
E = b_hat*(P_bar/Q_hat)
E_M = c_hat*(M_bar/Q_hat)
E_XAl = d_hat*(P_Al_bar/Q_hat)
E_XBMac = e_hat*(P_BMac_bar/Q_hat)

print('E = ',E,
      '\nE_M = ',E_M,
      '\nE_XAl = ',E_XAl,
      '\nE_XBMac = ',E_XBMac)

E =  -0.6933209981256101 
E_M =  0.8705356509822103 
E_XAl =  0.36810807211162117 
E_XBMac =  0.029562664219400972


The results tell us something interesting:

- At that price, the <font color='blue'>_**demand is inelastic**_</font>. In other words, if the price varies, the percentage change in demand will be smaller than that in price.

- A 10% increase in average household income will cause sales to rise by 8.71% - pizzas are a <font color='blue'>_**normal good**_</font> here.

- If Al's Pizza raises its pizza price in 10%, sales of Checkers' Pizza will increase by 3.68%.

- The effect of Big Mac's price in sales of Checkers' Pizza is quite small: if Big Mac's price decrease by 10%, the sales of Checkers' Pizza decrease by 0.29%.

---
### Final considerations

It is possible apply this quantity and elasticity analysis in different points along the demand curve.

It is also important to remember that this is an estimate of demand for a single price-setting firm, which is considerably simpler than estimating industry demand.

Furthermore, there are some precautions that are necessary when using this method and its estimates, such as the correct specification of the model and considerations about confidence intervals.

Of course, in the regression literature, there are many other interesting aspects of these methods and other relevant points about the care required when using them.

I hope it help you!